In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
import numpy as np
import json

In [2]:
clean_ranking_df = pd.read_csv(os.path.join('data','clean_ranking_v2.csv'), index_col=0)

/Users/Fer/anaconda3/envs/spotify_env/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
clean_ranking_df.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,song_id,release_date
2176,177,Footloose,Kenny Loggins,24993,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11,Canada,4O4Z8VFczL8MxIOmqVWc1b,2018-01-11
3392,193,Tequila,Dan + Shay,23917,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17,Canada,7Il2yWQmsqKgSkLQvHTPUI,2018-01-17
3594,195,Mine,Bazzi,23780,6tHWl8ows5JOZq9Yfaqn3M,2018-01-18,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15
3772,173,Mine,Bazzi,26500,6tHWl8ows5JOZq9Yfaqn3M,2018-01-19,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15
3978,179,Mine,Bazzi,24577,6tHWl8ows5JOZq9Yfaqn3M,2018-01-20,Canada,6tHWl8ows5JOZq9Yfaqn3M,2018-01-15


In [24]:
def generate_song_sequences_fn(song_ranking):
    dates=song_ranking.groupby('Region')['Date'].min().to_frame()
    dates_sort=dates.sort_values('Date')
    dates_sort= dates_sort.reset_index().set_index('Date')
    
    dates_lst=dates_sort.index.unique().values
    l=None
    if len(dates_lst)>= 2:
        l=dates_sort.loc[dates_lst[0]].values
        l=np.array(l).flatten()
        
        print(l)

        len_=1
        for d in range(1,len(dates_lst)):
            l2=dates_sort.loc[dates_lst[d]].values
            l2=np.array(l2).flatten()
            
            if len_ < 2:
                l=[[a,b] for a in l for b in l2]
            else:
                l=[a+[b] for a in l for b in l2]            
            len_+=1
    
            """
            if len_ < 2:
                l=[(a,)+(b,) for a in l for b in l2]
            else:
                l=[a+(b,) for a in l for b in l2]
            len_+=1
            """
    return l

In [ ]:
def generate_song_direct_sequences_fn(song_ranking, influence_threshold = 3, min_threshold=1):
    
    dates=song_ranking.groupby('Region')['Date'].min().to_frame()
    dates_sort=dates.sort_values('Date')
    dates_sort= dates_sort.reset_index().set_index('Date')
    
    dates_lst=dates_sort.index.unique().values
    
    i_start = 0
    list_ = []
    if len(dates_lst)>= 2:
        for i_start in range(0,len(dates_lst)):
            do =dates_lst[i_start]
            lo=dates_sort.loc[do].values
            lo=np.array(lo).flatten()

            for i_end in range(i_start+1,len(dates_lst)):
                de =dates_lst[i_end]
                dif = int((de-do)/86400000000000)

                if (dif <= influence_threshold) and (dif> min_threshold):
                    if dif>0:
                        #print("comparamos", dif, dates_lst[i_start], dates_lst[i_end])

                        le=dates_sort.loc[de].values
                        le=np.array(le).flatten()

                        l =[[a,b] for a in lo for b in le]
                        #print(l)
                        list_ = list_+l
    return list_

In [ ]:
seq={}
influence_threshold = 3 #days
min_threshold=1 #days
for s in tqdm(clean_ranking_df['song_id'].unique()):
    song_ranking= clean_ranking_df[clean_ranking_df['song_id']==s]
    song_seq = generate_song_direct_sequences_fn(song_ranking, influence_threshold, min_threshold)
    if (song_seq is not None) and len(song_seq)>1:
        seq[s]=song_seq

In [32]:
with open(os.path.join('data',f'clean_song_sequence_direct_{min_threshold}_{influence_threshold}.json'), 'w') as json_file:
    json.dump(seq, json_file)

Generate ranking with consequtive secuences

In [33]:
seq={}
influence_threshold = 5 #days
min_threshold=0 #days
max_pos=4
ranking_lite_df = clean_ranking_df[clean_ranking_df['Position']<=max_pos]
for s in tqdm(ranking_lite_df['song_id'].unique()):
    song_ranking= ranking_lite_df[ranking_lite_df['song_id']==s]
    song_seq = generate_song_direct_sequences_fn(song_ranking, influence_threshold, min_threshold)
    if (song_seq is not None) and len(song_seq)>1:
        seq[s]=song_seq
        
with open(os.path.join('data',f'clean_song_sequence_direct_{min_threshold}_{influence_threshold}_{max_pos}.json'), 'w') as json_file:
    json.dump(seq, json_file)    

In [ ]:
print("That's all folks!")